In [1]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns

import json

In [2]:
def filtering_company(company):
    if pd.isna(company):
        return np.nan
    
    original = company
    company = company.replace('(농업회사법인유한회사)', '')
    company = company.replace('의료법인)', '')
    company = company.replace('농업회사법인', '')
    company = company.replace('가부시키가이샤', '')
    company = company.replace('카부시키카이샤', '')
    company = company.replace('(유한회사)', '')
    company = company.replace('주식회사', '')
    company = company.replace('㈜', '')
    company = company.replace('(주)', '')
    company = company.replace('（주）', '')
    company = company.replace('( 주 )', '')
    company = company.replace('( 주)', '')
    company = company.replace('(주 )', '')
    company = company.replace('주)', '')
    company = company.replace('주 )', '')
    company = company.replace('(유)', '')
    company = company.replace('유)', '')
    company = company.replace('우)', '')
    company = company.replace('(사)', '')
    company = company.replace('사)', '')
    company = company.replace('  ', ' ') # 신정보개발주식회사 ~~
    company = company.split('|')[0] # 상위 -> 하위

    try:
        
        length = len(company)
        index = set(range(length))

        pairs1 = list()
        stack = list()
        for i in range(length):
            if company[i] == '(':
                stack.append(i)
            elif company[i] == ')':
                if stack:
                    pairs1.append((stack.pop(), i))
                else:
                    if pairs1:
                        pairs1.append((pairs1.pop()[0], i))
                    else:
                        pairs1.append((i, i))
        
        if stack:
            pairs1.append((stack[0], length-1))
        
        pairs2 = list()
        stack = list()
        for i in range(length):
            if company[i] == '[':
                stack.append(i)
            elif company[i] == ']':
                if stack:
                    pairs2.append((stack.pop(), i))
                else:
                    if pairs2:
                        pairs2.append((pairs2.pop()[0], i))
                    else:
                        pairs2.append((i, i))
                        
        if stack:
            pairs2.append((stack[0], length-1))
            
        for start, end in pairs1 + pairs2:
            index -= set(range(start, end+1))

        index = list(index)
        index.sort()
        company = ''.join([company[idx] for idx in index])
        company = company.strip()
    
    except:
        print(original, '--->', company)
        
    return company

In [3]:
with open('../data/krx_volume_info.json', "r") as krx_volume_file:
    krx_volume = json.load(krx_volume_file)
krx_volume = pd.DataFrame(krx_volume).T.reset_index()
krx_volume = krx_volume.rename(columns={'index':'회사명', 'stock_code': '종목코드'})
dart = pd.read_csv("../data/company_data/dart_comp_22 (1).csv")
dart['종목코드'] = dart['종목코드'].apply(lambda x: '{:0>6}'.format(x))

In [4]:
# dart에 있는 종목 코드를 모두 krx_compay가 갖고 있는지 확인
krx_company = krx_volume['종목코드'].values
check = True
for idx, company in dart.iterrows():
    if company['종목코드'] not in krx_company:
        print(company)
        check = False
        break
if check:
    print('dart의 종목 코드가 KRX에 모두 들어가있습니다')

dart의 종목 코드가 KRX에 모두 들어가있습니다


In [5]:
dart_krx_merged = pd.merge(dart, krx_volume, how='inner', on=['종목코드'])
dart_krx_merged = dart_krx_merged[dart_krx_merged['volume'] != '-']
dart_krx_merged = dart_krx_merged.reset_index()
dart_krx_merged['회사명'] = dart_krx_merged['회사명_x'].apply(filtering_company)

In [6]:
com_2_biz = pd.read_csv('../data/company_data/company2biz.tsv', sep='\t')
dart_krx_merged = dart_krx_merged.merge(com_2_biz, on='회사명')

In [7]:
dart_krx_merged = dart_krx_merged[dart_krx_merged['매출액'] != 0]
# 비상장에서 가장 매출액이 큰 회사
dart_krx_merged = dart_krx_merged[dart_krx_merged['매출액'] < 475158632000].reset_index(drop=True)
dart_krx_merged = dart_krx_merged.drop(columns=['index', '회사명_y'])

In [8]:
total_stock = pd.read_csv('./230725시총.csv')
total_stock

,날짜,시가총액,종목코드,회사명
0,2023-07-25,120856238580,60310.0,3S
1,2023-07-25,196419527525,95570.0,AJ네트웍스
2,2023-07-25,303336206850,265520.0,AP시스템
3,2023-07-25,174954726400,211270.0,AP위성
4,2023-07-25,2760239788200,282330.0,BGF리테일
...,...,...,...,...
2264,2023-07-25,75600000000,24060.0,흥구석유
2265,2023-07-25,81576247520,10240.0,흥국
2266,2023-07-25,103354904525,189980.0,흥국에프엔비
2267,2023-07-25,520519906335,3280.0,흥아해운


In [9]:
dart_krx_merged['주가총액'] = dart_krx_merged['발행주식의 총수'] * dart_krx_merged['주가']

In [10]:
dart_krx_merged

,회사명_x,자산총계,부채총액,자본총계,매출액,당기순이익,영업이익(EBIT),법인유형,화폐,결산 월,...,순부채,상각전영업이익(EBITDA),상장분류,종목코드,주가,isuCd,volume,회사명,사업자번호,주가총액
0,3S,6.267539e+10,2.613063e+10,3.654476e+10,2.226571e+10,4.821952e+08,1.153521e+09,1,원,3.0,...,1.982424e+10,3.108356e+09,True,060310,3510.0,KR7060310000,-0.1165,3S,1298116215,1.624130e+11
1,AP위성,1.092994e+11,2.394835e+10,8.535104e+10,5.360413e+10,4.077471e+09,4.494271e+09,1,원,12.0,...,-4.833856e+10,4.999621e+09,True,211270,10050.0,KR7211270004,-0.1129,AP위성,1198638269,1.515772e+11
2,BYC,6.573446e+11,1.457968e+11,5.115477e+11,1.681693e+11,3.565014e+10,2.445795e+10,0,원,12.0,...,9.208871e+10,2.445795e+10,True,001460,366000.0,KR7001460005,-0.0889,BYC,1188103063,3.074400e+11
3,CBI,7.159259e+10,2.872329e+10,4.286931e+10,3.362946e+10,2.927886e+10,4.809387e+09,1,원,12.0,...,2.285238e+10,4.809387e+09,True,013720,408.0,KR7013720008,-0.1176,CBI,1398116250,1.210759e+11
4,CJ 바이오사이언스,8.848905e+10,3.783897e+10,5.065009e+10,3.836756e+09,3.382798e+10,3.136051e+10,1,원,12.0,...,-1.026603e+10,3.486407e+10,True,311690,29000.0,KR7311690002,-0.1215,CJ 바이오사이언스,1198623145,1.704815e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1378,흥구석유,9.393462e+10,1.384832e+10,8.008630e+10,1.467177e+11,2.657026e+09,1.360244e+09,1,원,12.0,...,1.332279e+10,1.360244e+09,True,024060,5680.0,KR7024060006,-0.1121,흥구석유,5018100268,8.520000e+10
1379,흥국,9.382169e+10,1.361705e+10,8.020464e+10,1.042731e+11,1.175305e+10,1.164486e+10,1,원,12.0,...,-4.459553e+09,1.316224e+10,True,010240,5600.0,KR7010240000,-0.1112,흥국,6058112557,6.900710e+10
1380,흥국에프엔비,1.938309e+11,1.074053e+11,8.642561e+10,7.065651e+10,1.042024e+10,8.896088e+09,1,원,12.0,...,8.853343e+10,8.896088e+09,True,189980,2670.0,KR7189980006,-0.1127,흥국에프엔비,1278610281,1.071680e+11
1381,흥아해운,2.929999e+11,1.649748e+11,1.280251e+11,1.460296e+11,1.394174e+10,2.506706e+10,0,원,12.0,...,1.198348e+11,2.506706e+10,True,003280,1355.0,KR7003280005,-0.1038,흥아해운,1208162522,3.257757e+11


In [11]:
def stock_code_formatter(stock_code):
    return f'{int(stock_code):0>6}'
total_stock['종목코드'] = total_stock['종목코드'].apply(stock_code_formatter)

In [12]:
total_stock

,날짜,시가총액,종목코드,회사명
0,2023-07-25,120856238580,060310,3S
1,2023-07-25,196419527525,095570,AJ네트웍스
2,2023-07-25,303336206850,265520,AP시스템
3,2023-07-25,174954726400,211270,AP위성
4,2023-07-25,2760239788200,282330,BGF리테일
...,...,...,...,...
2264,2023-07-25,75600000000,024060,흥구석유
2265,2023-07-25,81576247520,010240,흥국
2266,2023-07-25,103354904525,189980,흥국에프엔비
2267,2023-07-25,520519906335,003280,흥아해운


In [13]:
dart_krx_merged = dart_krx_merged.merge(total_stock, on='회사명')

In [14]:
dart_krx_merged['volume'] = dart_krx_merged['volume'].astype(float)
dart_krx_merged[['부채총액', '자본총계', '매출액', '당기순이익', '영업이익(EBIT)', '현금', '감가상각비', '순부채', '상각전영업이익(EBITDA)', '시가총액']].corr()

,부채총액,자본총계,매출액,당기순이익,영업이익(EBIT),현금,감가상각비,순부채,상각전영업이익(EBITDA),시가총액
부채총액,1.000000,0.663121,0.411143,0.412539,0.559052,0.284032,0.233854,0.981424,0.574612,0.373119
자본총계,0.663121,1.000000,0.312877,0.407200,0.626018,0.372261,0.117470,0.614004,0.618136,0.513080
매출액,0.411143,0.312877,1.000000,0.219489,0.444724,0.327785,0.265859,0.361286,0.471092,0.224803
당기순이익,0.412539,0.407200,0.219489,1.000000,0.600114,0.283941,0.082759,0.371508,0.587327,0.321479
영업이익(EBIT),0.559052,0.626018,0.444724,0.600114,1.000000,0.391579,0.173874,0.502088,0.984996,0.534106
현금,0.284032,0.372261,0.327785,0.283941,0.391579,1.000000,0.106081,0.094807,0.392363,0.311743
감가상각비,0.233854,0.117470,0.265859,0.082759,0.173874,0.106081,1.000000,0.221575,0.341214,0.056022
순부채,0.981424,0.614004,0.361286,0.371508,0.502088,0.094807,0.221575,1.000000,0.518086,0.325016
상각전영업이익(EBITDA),0.574612,0.618136,0.471092,0.587327,0.984996,0.392363,0.341214,0.518086,1.000000,0.519636
시가총액,0.373119,0.513080,0.224803,0.321479,0.534106,0.311743,0.056022,0.325016,0.519636,1.000000


In [15]:
dart_krx_merged.drop(columns=['회사명']).rename(columns={'회사명_x':'회사명'}).to_csv('dart_volume.csv', index=False)

In [16]:
dart_krx_merged.head()

,회사명_x,자산총계,부채총액,자본총계,매출액,당기순이익,영업이익(EBIT),법인유형,화폐,결산 월,...,종목코드_x,주가,isuCd,volume,회사명,사업자번호,주가총액,날짜,시가총액,종목코드_y
0,3S,6.267539e+10,2.613063e+10,3.654476e+10,2.226571e+10,4.821952e+08,1.153521e+09,1,원,3.0,...,060310,3510.0,KR7060310000,-0.1165,3S,1298116215,1.624130e+11,2023-07-25,120856238580,060310
1,AP위성,1.092994e+11,2.394835e+10,8.535104e+10,5.360413e+10,4.077471e+09,4.494271e+09,1,원,12.0,...,211270,10050.0,KR7211270004,-0.1129,AP위성,1198638269,1.515772e+11,2023-07-25,174954726400,211270
2,BYC,6.573446e+11,1.457968e+11,5.115477e+11,1.681693e+11,3.565014e+10,2.445795e+10,0,원,12.0,...,001460,366000.0,KR7001460005,-0.0889,BYC,1188103063,3.074400e+11,2023-07-25,260152147500,001460
3,CBI,7.159259e+10,2.872329e+10,4.286931e+10,3.362946e+10,2.927886e+10,4.809387e+09,1,원,12.0,...,013720,408.0,KR7013720008,-0.1176,CBI,1398116250,1.210759e+11,2023-07-25,48057529640,013720
4,CJ 바이오사이언스,8.848905e+10,3.783897e+10,5.065009e+10,3.836756e+09,3.382798e+10,3.136051e+10,1,원,12.0,...,311690,29000.0,KR7311690002,-0.1215,CJ 바이오사이언스,1198623145,1.704815e+11,2023-07-25,109166939040,311690


In [17]:
cols = ['부채총액', '자본총계', '매출액', '당기순이익', '영업이익(EBIT)', '현금', '감가상각비', '순부채', '상각전영업이익(EBITDA)']
X = dart_krx_merged[cols]
y = dart_krx_merged['시가총액']
X.head()

,부채총액,자본총계,매출액,당기순이익,영업이익(EBIT),현금,감가상각비,순부채,상각전영업이익(EBITDA)
0,2.613063e+10,3.654476e+10,2.226571e+10,4.821952e+08,1.153521e+09,6.306392e+09,1.954835e+09,1.982424e+10,3.108356e+09
1,2.394835e+10,8.535104e+10,5.360413e+10,4.077471e+09,4.494271e+09,7.228690e+10,5.053500e+08,-4.833856e+10,4.999621e+09
2,1.457968e+11,5.115477e+11,1.681693e+11,3.565014e+10,2.445795e+10,5.370814e+10,0.000000e+00,9.208871e+10,2.445795e+10
3,2.872329e+10,4.286931e+10,3.362946e+10,2.927886e+10,4.809387e+09,5.870906e+09,0.000000e+00,2.285238e+10,4.809387e+09
4,3.783897e+10,5.065009e+10,3.836756e+09,3.382798e+10,3.136051e+10,4.810499e+10,3.503555e+09,-1.026603e+10,3.486407e+10


In [18]:
y.sort_values()

699       9375042680
992      11610352250
982      15480000000
442      19208323080
703      19826911823
           ...      
25     4897571270000
35     5008696121200
78     5635948976400
283    6576345289600
135    8759750583300
Name: 시가총액, Length: 1372, dtype: int64

In [19]:
from scipy.stats import skew, kurtosis

print(f'skew:{skew(y)}')

skew:8.969193592646603


In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

cols = ['부채총액', '자본총계', '매출액', '당기순이익', '영업이익(EBIT)', '현금', '감가상각비', '순부채', '상각전영업이익(EBITDA)']
X = dart_krx_merged[cols]
y = dart_krx_merged['시가총액']

train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

train_ds = lgb.Dataset(train_x, label=train_y)
test_ds = lgb.Dataset(test_x, label=test_y)

param = {'objective': 'regression'}
param['metric'] = ['l1']
model = lgb.train(param, train_ds, 1000, test_ds, verbose_eval=100, early_stopping_rounds=100)
# lgb.cv(param, train_ds, 10, nfold=5)

In [ ]:
smes = pd.read_csv("../data/company_data/smes_comp_22.csv")

company_bis_no = pd.read_csv('../data/company_data/smes_회사명_사업자번호.csv')
company_bis_no = company_bis_no[['회사명', '사업자번호']]
smes = pd.merge(smes, company_bis_no, how='inner', on='회사명')

cols = ['부채총액', '자본총계', '매출액', '당기순이익', '영업이익(EBIT)', '현금', '감가상각비', '순부채', '상각전영업이익(EBITDA)']
X = smes[cols]
predict = model.predict(X)
smes['predict'] = predict

In [ ]:
from sklearn.metrics import mean_absolute_error
kotc_value = pd.read_csv('./kotc_value.csv')
kotc_value = kotc_value.rename(columns={'name':'회사명'})
kotc_value['회사명'] = kotc_value['회사명'].apply(filtering_company)
kotc_value['value'] = kotc_value['value'] * 100000000
smes['회사명'] = smes['회사명'].apply(filtering_company)
test_metric = smes[['회사명', 'predict']].rename(columns={'predict':'value'}).merge(kotc_value, on='회사명')
mean_absolute_error(test_metric['value_x'], test_metric['value_y'])

In [ ]:
# from scipy.special import boxcox, inv_boxcox
import scipy

l = dart_krx_merged['시가총액']
boxcox_lambda = scipy.stats.boxcox(l - min(l) + 1)[1]
t_y = pd.Series(scipy.special.boxcox(l - min(l) + 1, boxcox_lambda))
print(t_y.skew())

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

cols = ['부채총액', '자본총계', '매출액', '당기순이익', '영업이익(EBIT)', '현금', '감가상각비', '순부채', '상각전영업이익(EBITDA)']
X = dart_krx_merged[cols]
y = t_y
# y = dart_krx_merged['volume']

train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

train_ds = lgb.Dataset(train_x, label=train_y)
test_ds = lgb.Dataset(test_x, label=test_y)


param = {'objective': 'regression'}
param['metric'] = ['l1']
model = lgb.train(param, train_ds, 1000, test_ds, verbose_eval=100, early_stopping_rounds=100)

In [ ]:
smes = pd.read_csv("./final_project/data/smes_comp_22.csv")

company_bis_no = pd.read_csv('./final_project/data/smes_회사명_사업자번호.csv')
company_bis_no = company_bis_no[['회사명', '사업자번호']]
smes = pd.merge(smes, company_bis_no, how='inner', on='회사명')

cols = ['부채총액', '자본총계', '매출액', '당기순이익', '영업이익(EBIT)', '현금', '감가상각비', '순부채', '상각전영업이익(EBITDA)']
X = smes[cols]
predict = model.predict(X)
predict = scipy.special.inv_boxcox(predict, boxcox_lambda) + min(l) - 1
# predict = np.power(10, predict) + min(l) - 1 # t_l = np.log10(l - min(l) + 1)
smes['predict'] = predict

In [ ]:
print(max(predict), min(predict))

In [ ]:
compare = pd.read_csv('./kotc_value.csv')
compare.head()

In [ ]:
smes['회사명'] = smes['회사명'].apply(filtering_company)

In [ ]:
compare = compare.rename(columns={'name':'회사명'})
smes_merge = smes.merge(compare, on='회사명')

In [ ]:
smes_merge['value'] = smes_merge['value'] * 100000000

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(smes_merge['value'], smes_merge['predict'])

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import  LabelEncoder
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
import re
import optuna
from optuna.integration import XGBoostPruningCallbackimport pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import  LabelEncoder
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
import re
import optuna
from optuna.integration import XGBoostPruningCallback

# Train

In [ ]:
dart_krx_merged = pd.read_csv('./dart_krx_merged.csv')

In [ ]:
# from scipy.special import boxcox, inv_boxcox
import scipy

l = dart_krx_merged['시가총액']
boxcox_lambda = scipy.stats.boxcox(l - min(l) + 1)[1]
t_y = pd.Series(scipy.special.boxcox(l - min(l) + 1, boxcox_lambda))
print(t_y.skew())

In [ ]:
cols = ['부채총액', '자본총계', '매출액', '당기순이익', '영업이익(EBIT)', '현금', '감가상각비', '순부채', '상각전영업이익(EBITDA)']
train_X = dart_krx_merged[cols]
train_Y = t_y

In [ ]:
def RMSE(y, y_pred):
    rmse = mean_squared_error(y, y_pred) ** 0.5
    return rmse

def rmse_cv(model):
    # cv별로 학습하는 함수
    tscv = TimeSeriesSplit(n_splits=10)
    rmse_list = []
    model_name = model.__class__.__name__
    for _, (train_index, test_index) in tqdm(enumerate(tscv.split(train_X), start=1), desc=f'{model_name} Cross Validations...', total=10):
        X_train, X_test = train_X.iloc[train_index], train_X.iloc[test_index]
        y_train, y_test = train_y.iloc[train_index], train_y.iloc[test_index]
        clf = model.fit(X_train, y_train)
        pred = clf.predict(X_test)
        rmse = RMSE(y_test, pred) 
        rmse_list.append(rmse)
    return model_name, rmse_list

def print_rmse_score(model):
    # cv별 프린팅, 평균 저장
    model_name, score = rmse_cv(model)
    # for i, r in enumerate(score, start=1):
    #     print(f'{i} FOLDS: {model_name} RMSLE: {r:.4f}')
    # print(f'\n{model_name} mean RMSLE: {np.mean(score):.4f}')
    # print('='*40)
    return model_name, np.mean(score)

In [ ]:
reg = LinearRegression(n_jobs=-1)
ridge = Ridge(alpha=0.8, random_state=1)
lasso = Lasso(alpha = 0.01, random_state=1)
Enet = ElasticNet(alpha=0.03, l1_ratio=0.01, random_state=1)
DTree = DecisionTreeRegressor(max_depth=6, min_samples_split=10, min_samples_leaf=15, random_state=1)
rf = RandomForestRegressor(n_estimators=500, criterion='mse', max_depth=9, min_samples_split=50,
                           min_samples_leaf=5, random_state=1, n_jobs=-1)
model_xgb = xgb.XGBRegressor(n_estimators=500, max_depth=9, min_child_weight=5, gamma=0.1, n_jobs=-1)
model_lgb = lgb.LGBMRegressor(n_estimators=500, max_depth=9, min_child_weight=5, n_jobs=-1)

In [ ]:
models = []
scores = []
for model in [reg, ridge, lasso, Enet, DTree, rf, model_xgb, model_lgb]:
    model_name, mean_score = print_rmse_score(model)
    models.append(model_name)
    scores.append(mean_score)

# Inference

In [ ]:
smes = pd.read_csv("./final_project/data/smes_comp_22.csv")

company_bis_no = pd.read_csv('./final_project/data/smes_회사명_사업자번호.csv')
company_bis_no = company_bis_no[['회사명', '사업자번호']]
smes = pd.merge(smes, company_bis_no, how='inner', on='회사명')

smes['회사명'] = smes['회사명'].apply(filtering_company)
cols = ['부채총액', '자본총계', '매출액', '당기순이익', '영업이익(EBIT)', '현금', '감가상각비', '순부채', '상각전영업이익(EBITDA)']
X = smes[cols]
for idx, model in enumerate([reg, ridge, lasso, Enet, DTree, rf, model_xgb, model_lgb]):
    predict = model.predict(X)
    predict = scipy.special.inv_boxcox(predict, boxcox_lambda) + min(l) - 1
    smes[f'predict_{models[idx]}'] = predict
    smes[['회사명', f'predict_{models[idx]}']].to_csv(f'./predict/boosting_{models[idx]}.csv', index=False)

In [ ]:
smes[['회사명', 'predict']].to_csv('./predict/boosting_rf.csv', index=False)

In [ ]:
# Hyperparameter tuning

In [ ]:
train_X

In [ ]:
cols = ['부채총액', '자본총계', '매출액', '당기순이익', '영업이익(EBIT)', '현금', '감가상각비', '순부채', '상각전영업이익(EBITDA)']
train_X = dart_krx_merged[cols]
train_Y = t_y

In [ ]:
import numpy as np
list(np.arange(0, 1, 0.1))

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV

gridParams = {
    'objective' : ['regression'],
    'learning_rate': [0.005, 0.01],
    'n_estimators': list(range(8, 513, 32)),
    'num_leaves': list(range(2, 257, 8)), # large num_leaves helps improve accuracy but might lead to over-fitting
    # 'boosting_type' : ['gbdt', 'dart'], # for better accuracy -> try dart
    'max_bin':[255, 510], # large max_bin helps improve accuracy but might slow down training progress
    'lambda_l1' : list(np.arange(0, 1, 0.1)),
    'lambda_l2' : list(np.arange(0, 1, 0.1)),
    'min_child_samples' : list(range(5, 101, 5)),
    'bagging_freq' : list(range(1, 8))
    }

grid = GridSearchCV(lgb.LGBMRegressor(verbose=-1), gridParams, cv=4, n_jobs=-1)
grid.fit(train_X, train_Y)

In [ ]:
smes = pd.read_csv('../data/company_data/smes_comp_22.csv')

company_bis_no = pd.read_csv('../data/company_data/smes_회사명_사업자번호.csv')
company_bis_no = company_bis_no[['회사명', '사업자번호']]
smes = pd.merge(smes, company_bis_no, how='inner', on='회사명')

smes['회사명'] = smes['회사명'].apply(filtering_company)
cols = ['부채총액', '자본총계', '매출액', '당기순이익', '영업이익(EBIT)', '현금', '감가상각비', '순부채', '상각전영업이익(EBITDA)']
X = smes[cols]

estimator = grid.best_estimator_
predict = estimator.predict(X)
predict = scipy.special.inv_boxcox(predict, boxcox_lambda) + min(l) - 1
smes['predict'] = predict

In [ ]:
smes.to_csv("./predict/grid_lgbm.csv", index=False)

In [ ]:
import sklearn
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV

def RMSE(y, y_pred):
    rmse = mean_squared_error(y, y_pred) ** 0.5
    return rmse

def objective(trial):
    train_x, test_x, train_y, test_y = train_test_split(train_X, train_Y, test_size=0.25)
    dtrain = lgb.Dataset(train_x, label=train_y)
 
    param = {
        'objective': 'regression',
        'verbose': -1,
        'metric': 'rmse',
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }
 
    gbm = lgb.train(param, dtrain)
    preds = gbm.predict(test_x)
    pred_labels = np.rint(preds)
    rmse = RMSE(test_y, pred_labels)
    return rmse
 
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)
 
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
study.best_value

In [ ]:
lgb.LGBMRegressor()

In [ ]:
trial = study.best_trial
trial_params = trial.params

In [ ]:
tuned_lgb_model = lgb.LGBMRegressor(**trial_params)
tuned_lgb_model.fit(train_X, train_Y)

In [ ]:
smes = pd.read_csv('../data/company_data/smes_comp_22.csv')

company_bis_no = pd.read_csv('../data/company_data/smes_회사명_사업자번호.csv')
company_bis_no = company_bis_no[['회사명', '사업자번호']]
smes = pd.merge(smes, company_bis_no, how='inner', on='회사명')

smes['회사명'] = smes['회사명'].apply(filtering_company)
cols = ['부채총액', '자본총계', '매출액', '당기순이익', '영업이익(EBIT)', '현금', '감가상각비', '순부채', '상각전영업이익(EBITDA)']
X = smes[cols]
predict = tuned_lgb_model.predict(X)
predict = scipy.special.inv_boxcox(predict, boxcox_lambda) + min(l) - 1
smes['predict'] = predict

In [ ]:
from sklearn.metrics import mean_absolute_error
kotc_value = pd.read_csv('./kotc_value.csv')
kotc_value = kotc_value.rename(columns={'name':'회사명'})
kotc_value['회사명'] = kotc_value['회사명'].apply(filtering_company)
kotc_value['value'] = kotc_value['value'] * 100000000
smes['회사명'] = smes['회사명'].apply(filtering_company)
test_metric = smes[['회사명', 'predict']].rename(columns={'predict':'value'}).merge(kotc_value, on='회사명')
mean_absolute_error(test_metric['value_x'], test_metric['value_y'])

In [ ]:
smes.to_csv('./predict/boosting_lgb_optuna.csv', index=False)